# Actividad 1

In [14]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cargar el conjunto de datos
archivo = 'Taller_2_Titulacion_DatosTaller.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

# Ver el número de valores faltantes por columna
valores_faltantes = datos.isnull().sum()
print("Valores faltantes por columna:")
print(valores_faltantes)

# Renombrar la columna 'Rotulo_Titulado' a 'Rotulo_Titulo' si es necesario
if 'Rotulo_Titulado' in datos.columns:
    datos = datos.rename(columns={'Rotulo_Titulado': 'Rotulo_Titulo'})

# Verificar si la columna 'Rotulo_Titulo' existe
if 'Rotulo_Titulo' not in datos.columns:
    raise ValueError("La columna 'Rotulo_Titulo' no se encuentra en el conjunto de datos.")

# Codificar la columna objetivo (si es necesario)
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Verificar los valores únicos en la columna objetivo
print("Valores únicos en 'Rotulo_Titulo':", datos['Rotulo_Titulo'].unique())

# Identificar las columnas categóricas
columnas_categoricas = datos.select_dtypes(include=['object']).columns

# Codificar las variables categóricas
datos_codificados = pd.get_dummies(datos, columns=columnas_categoricas, drop_first=True)

# Normalizar las características numéricas
columnas_numericas = datos_codificados.select_dtypes(include=['float64', 'int64']).columns
escalador = StandardScaler()
datos_codificados[columnas_numericas] = escalador.fit_transform(datos_codificados[columnas_numericas])

# Guardar los datos preprocesados para la siguiente actividad
datos_codificados.to_csv('datos_preprocesados.csv', index=False)

print("Preprocesamiento completo y datos guardados en 'datos_preprocesados.csv'.")

Valores faltantes por columna:
Id                                                     0
MAT_1SEM_PROM                                          0
FIS_1SEM_PROM                                          0
ING_1SEM_PROM                                          0
ACTF_1SEM_A                                            0
OTRANS_1SEM_A                                          0
OTRANS_1SEM_R                                          0
OTRANS_1SEM_PROM                                       0
ESP_1SEM_A                                             0
ESP_1SEM_R                                             0
ESP_1SEM_PROM                                          0
INS_1SEM                                               0
PROM_1SEM                                              0
MAT_2SEM_PROM                                          0
FIS_2SEM_PROM                                          0
ING_2SEM_PROM                                          0
ACTF_2SEM_A                                            0


# Actividad 2

In [36]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# Cargar el conjunto de datos preprocesado
archivo = 'datos_preprocesados.csv'
datos = pd.read_csv(archivo)

# Verificar y corregir la columna objetivo si es necesario
if datos['Rotulo_Titulo'].dtype == object:
    datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Asegurarse de que 'Rotulo_Titulo' sea de tipo entero
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].astype(int)

# Separar características y etiquetas
X = datos.drop('Rotulo_Titulo', axis=1)
y = datos['Rotulo_Titulo']

# Verificar nuevamente el tipo de y después de la codificación
print("Tipos únicos en 'Rotulo_Titulo':", y.unique())

# Definir el número de pliegues para la validación cruzada
num_pliegues = 5
validacion_cruzada = StratifiedKFold(n_splits=num_pliegues, shuffle=True, random_state=42)

# Iterar sobre los pliegues
for i, (train_index, val_index) in enumerate(validacion_cruzada.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    # Aquí puedes entrenar tu modelo usando X_train, y_train
    # y evaluarlo usando X_val, y_val para cada pliegue
    
    print(f"Pliegue {i+1}: Tamaño de entrenamiento = {len(train_index)}, Tamaño de validación = {len(val_index)}")


Tipos únicos en 'Rotulo_Titulo': [ 0 -1]
Pliegue 1: Tamaño de entrenamiento = 2414, Tamaño de validación = 604
Pliegue 2: Tamaño de entrenamiento = 2414, Tamaño de validación = 604
Pliegue 3: Tamaño de entrenamiento = 2414, Tamaño de validación = 604
Pliegue 4: Tamaño de entrenamiento = 2415, Tamaño de validación = 603
Pliegue 5: Tamaño de entrenamiento = 2415, Tamaño de validación = 603


# Actividad 3

In [38]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Cargar los datos preprocesados
archivo = 'datos_preprocesados.csv'
datos = pd.read_csv(archivo)

# Verificar y corregir la columna objetivo si es necesario
if datos['Rotulo_Titulo'].dtype == object:
    datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Asegurarse de que 'Rotulo_Titulo' sea de tipo entero
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].astype(int)

# Separar características y etiquetas
X = datos.drop('Rotulo_Titulo', axis=1)
y = datos['Rotulo_Titulo']

# Definir los hiperparámetros para SVM
parametros_svm = {
    'C': [1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3]
}

# Crear el modelo de SVM
modelo_svm = SVC()

# Ejecutar la búsqueda de hiperparámetros para SVM
grid_search_svm = GridSearchCV(modelo_svm, parametros_svm, cv=5, verbose=1, n_jobs=-1)
grid_search_svm.fit(X, y)

# Mostrar resultados
print("Mejor configuración encontrada para SVM:")
print(grid_search_svm.best_params_)
print("Puntaje de validación cruzada:")
print(grid_search_svm.best_score_)


Optimizando Random Forest...
Fitting 5 folds for each of 36 candidates, totalling 180 fits


c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
90 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
78 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.p

Mejor configuración encontrada: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 150}
Puntaje de validación cruzada: 0.7524655969600123

Optimizando SVM...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Mejor configuración encontrada: {'C': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Puntaje de validación cruzada: 0.7465080777129803

El mejor modelo encontrado es: Random Forest
Con una precisión media de: 0.7465080777129803


In [44]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Cargar los datos preprocesados
archivo = 'datos_preprocesados.csv'
datos = pd.read_csv(archivo)

# Verificar y corregir la columna objetivo si es necesario
if datos['Rotulo_Titulo'].dtype == object:
    datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Asegurarse de que 'Rotulo_Titulo' sea de tipo entero
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].astype(int)

# Separar características y etiquetas
X = datos.drop('Rotulo_Titulo', axis=1)
y = datos['Rotulo_Titulo']

# Definir los hiperparámetros para Random Forest
parametros_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2']
}

# Crear el modelo de Random Forest
modelo_rf = RandomForestClassifier()

# Ejecutar la búsqueda de hiperparámetros para Random Forest
grid_search_rf = GridSearchCV(modelo_rf, parametros_rf, cv=5, verbose=1, n_jobs=-1)
grid_search_rf.fit(X, y)

# Mostrar resultados
print("Mejor configuración encontrada para Random Forest:")
print(grid_search_rf.best_params_)
print("Puntaje de validación cruzada:")
print(grid_search_rf.best_score_)


Fitting 5 folds for each of 36 candidates, totalling 180 fits
Mejor configuración encontrada para Random Forest:
{'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100}
Puntaje de validación cruzada:
0.7557790517610623


# Actividad 4

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cargar el conjunto de datos
archivo = 'Taller_2_Titulacion_DatosTaller.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

# Ver el número de valores faltantes por columna
valores_faltantes = datos.isnull().sum()
print("Valores faltantes por columna:")
print(valores_faltantes)

# Renombrar la columna 'Rotulo_Titulado' a 'Rotulo_Titulo' si es necesario
if 'Rotulo_Titulado' in datos.columns:
    datos = datos.rename(columns={'Rotulo_Titulado': 'Rotulo_Titulo'})

# Verificar si la columna 'Rotulo_Titulo' existe
if 'Rotulo_Titulo' not in datos.columns:
    raise ValueError("La columna 'Rotulo_Titulo' no se encuentra en el conjunto de datos.")

# Codificar la columna objetivo (si es necesario)
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Verificar los valores únicos en la columna objetivo
print("Valores únicos en 'Rotulo_Titulo':", datos['Rotulo_Titulo'].unique())

# Identificar las columnas categóricas
columnas_categoricas = datos.select_dtypes(include=['object']).columns

# Codificar las variables categóricas
datos_codificados = pd.get_dummies(datos, columns=columnas_categoricas, drop_first=True)

# Normalizar las características numéricas
columnas_numericas = datos_codificados.select_dtypes(include=['float64', 'int64']).columns
escalador = StandardScaler()
datos_codificados[columnas_numericas] = escalador.fit_transform(datos_codificados[columnas_numericas])

# Guardar los datos preprocesados para la siguiente actividad
datos_codificados.to_csv('datos_preprocesados.csv', index=False)

print("Preprocesamiento completo y datos guardados en 'datos_preprocesados.csv'.")

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# Cargar el conjunto de datos preprocesado
archivo = 'datosEvaluacion_preprocesados.csv'
datos = pd.read_csv(archivo)

# Verificar y corregir la columna objetivo si es necesario
if datos['Rotulo_Titulo'].dtype == object:
    datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Asegurarse de que 'Rotulo_Titulo' sea de tipo entero
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].astype(int)

# Separar características y etiquetas
X = datos.drop('Rotulo_Titulo', axis=1)
y = datos['Rotulo_Titulo']

# Verificar nuevamente el tipo de y después de la codificación
print("Tipos únicos en 'Rotulo_Titulo':", y.unique())

# Definir el número de pliegues para la validación cruzada
num_pliegues = 5
validacion_cruzada = StratifiedKFold(n_splits=num_pliegues, shuffle=True, random_state=42)

# Iterar sobre los pliegues
for i, (train_index, val_index) in enumerate(validacion_cruzada.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    # Aquí puedes entrenar tu modelo usando X_train, y_train
    # y evaluarlo usando X_val, y_val para cada pliegue
    
    print(f"Pliegue {i+1}: Tamaño de entrenamiento = {len(train_index)}, Tamaño de validación = {len(val_index)}")


In [41]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Cargar el conjunto de datos de evaluación

archivo_evaluacion = 'Taller_2_Titulacion_DatosTaller.csv'
datos_evaluacion = pd.read_csv(archivo, encoding='latin-1', delimiter=';')


# Preprocesamiento similar al utilizado en el entrenamiento
# Asegúrate de aplicar las mismas transformaciones que en los datos de entrenamiento
# Aquí deberías tener el mismo código de preprocesamiento que usaste antes del ajuste del modelo

# Ajustar las columnas de los datos de evaluación si es necesario
# Verificar y eliminar columnas adicionales que no se usaron en el entrenamiento
columnas_entrenamiento = X_entrenamiento.columns.tolist()
datos_evaluacion = datos_evaluacion[columnas_entrenamiento]

# Suponiendo que ya tienes cargado y ajustado el mejor modelo de la Actividad 3
mejor_modelo = RandomForestClassifier(n_estimators=150, max_depth=None, min_samples_split=5, max_features='sqrt')

# Ajustar el modelo a los datos de entrenamiento
X_entrenamiento = datos.drop('Rotulo_Titulo', axis=1)  # Ajusta según tus datos de entrenamiento
y_entrenamiento = datos['Rotulo_Titulo']  # Ajusta según tus datos de entrenamiento
mejor_modelo.fit(X_entrenamiento, y_entrenamiento)

# Preprocesar los datos de evaluación de la misma manera que los datos de entrenamiento
# Esto puede incluir codificación de variables categóricas, normalización, etc.
# Asegúrate de aplicar cualquier transformación que hayas hecho en los datos de entrenamiento

# Hacer predicciones sobre los datos de evaluación
predicciones = mejor_modelo.predict(datos_evaluacion)

# Crear un DataFrame con los resultados de predicción
resultados = pd.DataFrame({
    'Id': datos_evaluacion['Id'],  # Ajusta según la columna de identificación en tus datos de evaluación
    'Prediccion': predicciones  # Debe contener 'SI' o 'NO' según lo definido en tu modelo
})

# Generar el archivo de salida en formato txt
archivo_salida = 'clasificacion_titulo4.txt'
resultados.to_csv(archivo_salida, sep='\t', index=False, header=False)

print(f"Archivo '{archivo_salida}' generado exitosamente.")


KeyError: "None of [Index(['Id', 'MAT_1SEM_PROM', 'FIS_1SEM_PROM', 'ING_1SEM_PROM', 'ACTF_1SEM_A',\n       'OTRANS_1SEM_A', 'OTRANS_1SEM_R', 'OTRANS_1SEM_PROM', 'ESP_1SEM_A',\n       'ESP_1SEM_R',\n       ...\n       'descripcion_situacion_ocupacional_padre_Otra situación',\n       'descripcion_situacion_ocupacional_padre_Sin Datos',\n       'descripcion_situacion_ocupacional_padre_Trabaja sólo ocasionalmente',\n       'descripcion_situacion_ocupacional_madre_Cesante',\n       'descripcion_situacion_ocupacional_madre_Dueno de casa',\n       'descripcion_situacion_ocupacional_madre_Jubilado o pensionado (no reincorporado)',\n       'descripcion_situacion_ocupacional_madre_No trabaja (enfermedad& edad avanzada)',\n       'descripcion_situacion_ocupacional_madre_Otra situación',\n       'descripcion_situacion_ocupacional_madre_Sin Datos',\n       'descripcion_situacion_ocupacional_madre_Trabaja sólo ocasionalmente'],\n      dtype='object', length=341)] are in the [columns]"